In [ ]:
# rb model and projections

In [1]:
import pandas as pd

In [2]:
stats = pd.read_csv("ffb_stats.csv")

In [7]:
feats = ['Player','Tm','Pos','Rushing Att','Rushing Yds', 'Rushing TD','Year','Pts','Tgt','Rec','Receiving Yds','Receiving TD']

In [8]:
ffb_rb = stats[stats['Pos'] == 'RB']

In [9]:
rb = ffb_rb[feats]

In [11]:
predictors = ['Rushing Att','Rushing Yds', 'Rushing TD','Tgt','Rec','Receiving Yds','Receiving TD']

In [13]:
train = rb[rb['Year'] < 2022]
test = rb[rb['Year'] == 2022]

In [14]:
from sklearn.linear_model import Ridge

rbm = Ridge(alpha=.1)

In [15]:
rbm.fit(train[predictors], train["Pts"])

Ridge(alpha=0.1)

In [16]:
predictions = rbm.predict(test[predictors])

In [17]:
predictions = pd.DataFrame(predictions, columns=["Pred Pts"], index=test.index)

In [18]:
comb = pd.concat([test[["Player", "Pts"]], predictions], axis=1)

In [19]:
comb

,Player,Pts,Pred Pts
159,Christian McCaffrey,266.0,253.063576
161,Josh Jacobs,265.0,261.669609
199,Austin Ekeler,251.0,261.192048
200,Derrick Henry,250.0,256.360616
208,Nick Chubb,249.0,240.840015
...,...,...,...
6976,Ty'Son Williams,0.0,-0.911160
7484,Trestan Ebner,-3.0,3.883189
7501,Mike Davis,-3.0,0.630974
7502,Giovani Bernard,-3.0,1.112188


In [20]:
from sklearn.metrics import mean_squared_error

In [21]:
mean_squared_error(comb["Pts"], comb["Pred Pts"])

16.457735221563446

In [22]:
import pickle

In [23]:
with open('rbm.pkl', 'wb') as model_file:
    pickle.dump(rbm, model_file)

In [37]:
rb_proj = pd.read_csv('rb_projections.csv')

In [38]:
del rb_proj ['Proj Pts']

In [39]:
rb_proj.columns

Index(['Player', 'Team', 'Ru Att', 'Ru Yds', 'Ru TDs', 'Tgt', 'Rec', 'Rec Yds',
       'Rec TDs'],
      dtype='object')

In [40]:
rb_proj.rename(columns={'Ru Att': 'Rushing Att'}, inplace=True)

In [41]:
rb_proj.rename(columns={'Ru Yds': 'Rushing Yds'}, inplace=True)

In [42]:
rb_proj.rename(columns={'Ru TDs': 'Rushing TD'}, inplace=True)

In [43]:
rb_proj.rename(columns={'Rec Yds': 'Receiving Yds'}, inplace=True)

In [44]:
rb_proj.rename(columns={'Rec TDs': 'Receiving TD'}, inplace=True)

In [45]:
rb_proj.rename(columns={'Team': 'Tm'}, inplace=True)

In [46]:
rb_proj

,Player,Tm,Rushing Att,Rushing Yds,Rushing TD,Tgt,Rec,Receiving Yds,Receiving TD
0,Christian McCaffrey,SF,205.2,953.8,7.1,81.80,66.55,566.4,3.80
1,Austin Ekeler,LAC,187.9,811.5,8.8,84.35,70.00,536.9,4.00
2,Bijan Robinson,ATL,262.9,1197.0,9.2,50.20,38.55,288.9,1.85
3,Saquon Barkley,NYG,261.2,1133.5,8.6,68.50,50.95,367.1,1.20
4,Nick Chubb,CLE,260.8,1292.4,9.4,37.35,29.40,252.6,1.10
...,...,...,...,...,...,...,...,...,...
114,Salvon Ahmed,MIA,14.8,64.2,0.3,2.80,1.50,11.0,0.10
115,Avery Williams,ATL,2.5,10.8,0.1,7.70,6.30,40.9,0.20
116,Patrick Ricard,BAL,3.3,12.0,NaN,7.90,6.30,43.3,0.20
117,La'Mical Perine,KC,9.4,34.4,0.2,6.10,1.90,14.5,0.10


In [47]:
rb_proj = rb_proj.fillna(0)

In [48]:
rb_pred = rbm.predict(rb_proj[predictors])

In [49]:
rb_pred = pd.DataFrame(rb_pred, columns=["Pred Pts"], index=rb_proj.index)

In [50]:
rb_comb = pd.concat([rb_proj[["Player", "Tm"]], rb_pred], axis=1)

In [51]:
rb_comb = rb_comb.sort_values("Pred Pts", ascending=False)
rb_comb["Pred Rk"] = list(range(1,rb_comb.shape[0]+1))

In [53]:
rb_comb.head(10)

,Player,Tm,Pred Pts,Pred Rk
0,Christian McCaffrey,SF,206.762027,1
4,Nick Chubb,CLE,205.605721,2
2,Bijan Robinson,ATL,203.379599,3
5,Derrick Henry,TEN,203.189578,4
1,Austin Ekeler,LAC,202.327339,5
6,Jonathan Taylor,IND,202.037183,6
3,Saquon Barkley,NYG,196.812088,7
7,Tony Pollard,DAL,190.161195,8
8,Josh Jacobs,LV,189.973463,9
9,Rhamondre Stevenson,NE,173.220155,10


In [54]:
rb_comb.to_csv('proj_rb.csv')